In [3]:
import pandas as pd
import glob
import os
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean


In [13]:
def metric(path, r_string, type_string, timing=False):
    resultdf = dict()

    all_files = glob.glob(os.path.join(path, "*"+r_string+type_string+"*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)
 
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        if name not in resultdf:
            resultdf[name] = dict()
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
        max_key = df.tail(1).index.values[0]
        gnn_iteration = df['Iteration'][max_key]
        gnn_cpd = df['CPD (ns)'][max_key]
        gnn_time = sum(df['Time'])

        resultdf[name]['GNN_Iterations'] = gnn_iteration
        resultdf[name]['GNN_CPD (ns)'] = gnn_cpd
        resultdf[name]['GNN_Time (s)'] = gnn_time

    all_files = glob.glob(os.path.join(path, r_string+"*reg*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)

    type_string = "reg"
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        df = pd.read_csv(reg_metric,index_col=None,header=0)
        max_key = df.tail(1).index.values[0]
        reg_iteration = df['Iteration'][max_key]
        reg_cpd = df['CPD (ns)'][max_key]
        reg_time = sum(df['Time'])

        resultdf[name]['REG_Iterations'] = reg_iteration
        resultdf[name]['REG_CPD (ns)'] = reg_cpd
        resultdf[name]['REG_Time (s)'] = reg_time
    
   
    resultdf = pd.DataFrame(resultdf).T
    resultdf['GNN_Iterations'] = resultdf['GNN_Iterations'].astype(int)
    resultdf['REG_Iterations'] = resultdf['REG_Iterations'].astype(int)
    resultdf = resultdf[["GNN_Iterations", "REG_Iterations", "GNN_Time (s)", "REG_CPD (ns)"]]
    pd.set_option('display.float_format','{:.2f}'.format)
    print(resultdf)

In [32]:
path = "/mnt/e/benchmarks/route_metrics/MCNC/"
r_string = "EArch*"
type_string = "gnn"
metric(path, r_string, type_string)

                 GNN_Iterations  REG_Iterations  GNN_Time (s)  REG_CPD (ns)
EArch__alu4                  26              20          3.80          6.03
EArch__apex2                112              -1          7.08          6.79
EArch__apex4                109              40          6.34          5.78
EArch__bigkey                10               6          4.91          2.45
EArch__clma                  -1              40         10.96         12.02
EArch__des                   -1              34          3.29          5.12
EArch__diffeq                -1               5          3.57          6.88
EArch__dsip                  -1               5          3.31          2.61
EArch__elliptic              -1              17          4.08          8.72
EArch__ex1010                10              59          5.86          7.72
EArch__ex5p                   9              33          4.31          5.44
EArch__frisc                 10              32          5.31         12.63
EArch__misex

In [6]:
import tensorflow as tf
import datetime

In [7]:
path = "/mnt/e/benchmarks/route_metrics/TITAN/"
r_string = "strat*"
type_string = "gnn"
metric(path, r_string, type_string)

KeyError: 'stratixiv_arch__uoft_raytracer_stratixiv_arch_timing'